In [1]:
from __future__ import division, print_function
import sys, os, glob, time, warnings, gc
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, vstack, hstack
import fitsio
# from astropy.io import fits

In [2]:
params = {'legend.fontsize': 'large',
         'axes.labelsize': 'large',
         'axes.titlesize':'large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large',
         'figure.facecolor':'w'} 
plt.rcParams.update(params)

In [3]:
indir = '/global/cfs/cdirs/desi/spectro/redux/fuji/exposures'
os.environ['INDIR'] = indir

overwrite = False

In [4]:
subsamp_dict = {'80605': [[74781, 73702],
  [67975, 74782],
  [68292, 74779],
  [68291, 68290],
  [74783, 74780]],
 '80606': [[68630, 67968], [67971, 68812], [67969, 68813, 67970]],
 '80607': [[67768, 68844],
  [68027, 68845],
  [67767, 68028, 67766],
  [68847, 67744],
  [67765, 68846]],
 '80608': [[68025, 68328],
  [67770, 68026],
  [67771, 68327],
  [67769, 68024],
  [68842, 68023],
  [68491, 68841]],
 '80609': [[68489, 67781],
  [67784, 68338],
  [68336, 68490],
  [68064, 68334],
  [68340, 67783],
  [68063, 68065]],
 '80610': [[68333, 68477],
  [68332, 68331],
  [68330, 68042],
  [68488, 68041],
  [68040, 75116]],

 '80620': [[68677, 68851], [68676, 68850]],
 '80678': [[71878, 72681],
  [74946, 74943],
  [72505, 71876],
  [72504, 71877],
  [74944, 74945]],
 '80690': [[71892, 76294],
  [75840, 74802, 75841],
  [75838, 72515, 72692],
  [75842, 75839, 76293],
  [74801, 72514],
  [74803, 76295]],
 '80695': [[80944, 79553], [79552, 80943]],
 '80699': [[72519, 71481], [72518, 71618], [79447, 71619]],
 '80711': [[75862, 75860, 74462, 75861],
  [75859, 72714, 86518],
  [74831, 74832],
  [72715, 74829, 74830]]}

In [5]:
# Print the total number of exposures used in the subsets
count = 0
for tileid in ['80605', '80607', '80609', '80606', '80608', '80610']:
    for tmp in subsamp_dict[tileid]:
        count += len(tmp)
print(count)

62


In [6]:
cat = Table(fitsio.read('/global/cfs/cdirs/desi/spectro/redux/fuji/exposures-fuji.fits', ext='EXPOSURES'))
print(len(cat), len(np.unique(cat['EXPID'])))

exp_list = sorted([i for k in subsamp_dict.values() for j in k for i in j])
print(len(exp_list), len(np.unique(exp_list)))

mask = np.in1d(cat['EXPID'], exp_list)
print(np.sum(mask))
cat = cat[mask]

2480 2480
113 113
113


In [7]:
explist = cat[['NIGHT', 'EXPID', 'TILEID']]
explist

NIGHT,EXPID,TILEID
int32,int32,int32
20201214,67744,80607
20201214,67765,80607
20201214,67766,80607
20201214,67767,80607
20201214,67768,80607
20201214,67769,80608
20201214,67770,80608
20201214,67771,80608
20201214,67781,80609


In [8]:
cframes_stack = []

for index in range(len(explist)):
    tileid = explist['TILEID'][index]
    night = explist['NIGHT'][index]
    expid = explist['EXPID'][index]
    exposure_dir = os.path.join(indir, str(night), str(expid).zfill(8))
    cframe_list = glob.glob(os.path.join(exposure_dir, 'cframe-*'))
    if len(cframe_list)==0:
        print(exposure_dir, 'is empty')
        continue

    cframes = Table()
    cframes['cframe_fn'] = np.array(cframe_list)
    cframes['tileid'] = np.zeros(len(cframes), dtype=int)
    cframes['expid'] = np.zeros(len(cframes), dtype=int)
    cframes['camera'] = ' '
    cframes['petal_loc'] = -1 * np.ones(len(cframes), dtype=np.int32)

    for cframe_index, cframe_fn in enumerate(cframe_list):
        _, cp, expid1 = os.path.basename(cframe_fn).strip('.fits').split('-')
        camera, petal_loc = cp[0], cp[1]
        expid1 = int(expid)
        if expid1!=expid:
            raise ValueError()
        cframes['tileid'][cframe_index] = tileid
        cframes['expid'][cframe_index] = expid
        cframes['camera'][cframe_index] = camera
        cframes['petal_loc'][cframe_index] = petal_loc
    
    cframes_stack.append(cframes)
    
cframes = vstack(cframes_stack)
print(len(cframes))
print(len(np.unique(cframes['expid'])))

cframes[:1]

3228
113


cframe_fn,tileid,expid,camera,petal_loc
str93,int64,int64,str1,int32
/global/cfs/cdirs/desi/spectro/redux/fuji/exposures/20201214/00067744/cframe-r1-00067744.fits,80607,67744,r,1


In [9]:
cframes['keep'] = False
for expid in np.unique(cframes['expid']):
    mask = cframes['expid']==expid
    for petal_loc in np.arange(10):
        mask1 = mask & (cframes['petal_loc']==petal_loc)
        if np.sum(mask1)==3:  # require all 3 cameras
            cframes['keep'][mask1] = True
print(np.sum(cframes['keep']), np.sum(~cframes['keep']))

cframes = cframes[cframes['keep']]
print(len(cframes))

3228 0
3228


In [10]:
# mask = cframes['tileid']==65008
# cframes = cframes[mask]

In [11]:
# keep only one camera for simplicity
mask = cframes['camera']=='b'
cframes = cframes[mask]
print(len(cframes))

cframes.sort('cframe_fn')

1076


In [12]:
cframes

cframe_fn,tileid,expid,camera,petal_loc,keep
str93,int64,int64,str1,int32,bool
/global/cfs/cdirs/desi/spectro/redux/fuji/exposures/20201214/00067744/cframe-b0-00067744.fits,80607,67744,b,0,True
/global/cfs/cdirs/desi/spectro/redux/fuji/exposures/20201214/00067744/cframe-b1-00067744.fits,80607,67744,b,1,True
/global/cfs/cdirs/desi/spectro/redux/fuji/exposures/20201214/00067744/cframe-b2-00067744.fits,80607,67744,b,2,True
/global/cfs/cdirs/desi/spectro/redux/fuji/exposures/20201214/00067744/cframe-b3-00067744.fits,80607,67744,b,3,True
/global/cfs/cdirs/desi/spectro/redux/fuji/exposures/20201214/00067744/cframe-b4-00067744.fits,80607,67744,b,4,True
/global/cfs/cdirs/desi/spectro/redux/fuji/exposures/20201214/00067744/cframe-b5-00067744.fits,80607,67744,b,5,True
/global/cfs/cdirs/desi/spectro/redux/fuji/exposures/20201214/00067744/cframe-b6-00067744.fits,80607,67744,b,6,True
/global/cfs/cdirs/desi/spectro/redux/fuji/exposures/20201214/00067744/cframe-b7-00067744.fits,80607,67744,b,7,True
/global/cfs/cdirs/desi/spectro/redux/fuji/exposures/20201214/00067744/cframe-b8-00067744.fits,80607,67744,b,8,True


------
## 1X depth coadds

Initial setup:
```
source $CFS/desi/software/desi_environment.sh 22.1b
module swap desispec/0.51.8
module swap redrock/0.15.3
export SPECPROD=fuji
export DESI_SPECTRO_CALIB=/global/cfs/cdirs/desi/spectro/desi_spectro_calib/0.3.1

export INDIR=/global/cfs/cdirs/desi/spectro/redux/fuji/exposures
export OUTDIR=/global/cfs/cdirs/desi/users/rongpu/fuji/1x_depth

module load parallel
```

Run desi_group_spectra:
```
salloc -N 20 -C haswell -q interactive -t 4:00:00
parallel --jobs 20 --delay 1 < desi_group_spectra_1x.txt ; exit
```

Run desi_coadd_spectra:
```
salloc -N 1 -C haswell -q interactive -t 4:00:00
parallel --jobs 16 < desi_coadd_spectra_1x.txt ; exit
```

Run redrock:
```
salloc -N 20 --qos interactive -C haswell -t 4:00:00
parallel --jobs 20 --delay 1 < rrdesi_1x.txt ; exit
```

Run QSO afterburners:

```
salloc -N 10 --qos interactive -C haswell -t 4:00:00
parallel --jobs 10 --delay 1 < qso_mgii_1x.txt
parallel --jobs 10 --delay 1 < qso_qn_1x.txt ; exit
```

Run emission line afterburners:

```
salloc -N 1 --qos interactive -C haswell -t 4:00:00
parallel --jobs 32 --delay 1 < desi_emlinefit_afterburner_1x.txt ; exit
```

In [13]:
# output_dir = os.getenv('OUTDIR')
outdir = os.path.expandvars('/global/cfs/cdirs/desi/users/rongpu/fuji/1x_depth')
os.environ['OUTDIR'] = outdir

__Print desi_group_spectra commands__

In [14]:
command_output_path_all = 'desi_group_spectra_1x_all.txt'
command_output_path = 'desi_group_spectra_1x.txt'  # commands for reruns (e.g. after failures)

f_all = open(command_output_path_all, 'w')
f = open(command_output_path, 'w')

for tileid_str in subsamp_dict.keys():
    
    # tileid_str = '80605'
    tileid = int(tileid_str)

    for subset_index, subset in enumerate(subsamp_dict[tileid_str]):
        # print(subset)

        mask = np.in1d(cframes['expid'], subset)
        _, n_exp = np.unique(cframes['petal_loc'][mask], return_counts=True)
        n_exp = np.max(n_exp)
        petal_locs = np.arange(10)
        
        for petal_loc in petal_locs:

            mask = np.in1d(cframes['expid'], subset) & (cframes['petal_loc']==petal_loc)
            if np.sum(mask)!=n_exp:
                print('Tile {} subset {} petal {} has only {} out of {} exposures; skip'.format(tileid, subset_index+1, petal_loc, np.sum(mask), n_exp))
                continue
        
            input_locs = []
            for expid in subset:
                mask = (cframes['expid']==expid) & (cframes['petal_loc']==petal_loc)
                if np.sum(mask)>1:
                    raise ValueError
                if np.sum(mask)==1:
                    exposure_dir = os.path.dirname(cframes['cframe_fn'][mask][0])
                    input_locs.append(os.path.join(exposure_dir.replace(indir, '$INDIR'), 'cframe-[brz]{}-*.fits'.format(petal_loc)))

            input_locs = ' '.join(input_locs)
            spectra_output_loc = os.path.join('$OUTDIR', str(tileid), str(subset_index+1), 'spectra-{}-{}-1xsubset{}.fits'.format(petal_loc, tileid, subset_index+1))
            
            write_text = 'srun -N 1 -n 1 -c 64 desi_group_spectra --inframes {} --outfile {}'.format(input_locs, spectra_output_loc)
            write_text += ' --header SPGRP=1x_depth SPGRPVAL={} TILEID={} SPECTRO={} PETAL={}'.format(subset_index+1, tileid, petal_loc, petal_loc)
            write_text += '\n'
            
            f_all.write(write_text)

            if os.path.isfile(os.path.expandvars(spectra_output_loc)) and (not overwrite):
                # print('{} already exists'.format(spectra_output_loc))
                pass
            else:
                # print(write_text)
                f.write(write_text)
                
f_all.close()
f.close()

Tile 80605 subset 1 petal 3 has only 0 out of 2 exposures; skip
Tile 80605 subset 2 petal 3 has only 1 out of 2 exposures; skip
Tile 80605 subset 3 petal 3 has only 1 out of 2 exposures; skip
Tile 80605 subset 5 petal 3 has only 0 out of 2 exposures; skip
Tile 80606 subset 1 petal 3 has only 1 out of 2 exposures; skip
Tile 80610 subset 5 petal 3 has only 1 out of 2 exposures; skip
Tile 80678 subset 1 petal 3 has only 0 out of 2 exposures; skip
Tile 80678 subset 2 petal 3 has only 0 out of 2 exposures; skip
Tile 80678 subset 3 petal 3 has only 0 out of 2 exposures; skip
Tile 80678 subset 4 petal 3 has only 0 out of 2 exposures; skip
Tile 80678 subset 5 petal 3 has only 0 out of 2 exposures; skip
Tile 80690 subset 1 petal 3 has only 0 out of 2 exposures; skip
Tile 80690 subset 2 petal 3 has only 0 out of 3 exposures; skip
Tile 80690 subset 3 petal 3 has only 0 out of 3 exposures; skip
Tile 80690 subset 4 petal 3 has only 0 out of 3 exposures; skip
Tile 80690 subset 5 petal 3 has only 0 o

__Print desi_coadd_spectra commands__

In [15]:
command_output_path_all = 'desi_coadd_spectra_1x_all.txt'
command_output_path = 'desi_coadd_spectra_1x.txt'  # commands for reruns (e.g. after failures)

f_all = open(command_output_path_all, 'w')
f = open(command_output_path, 'w')

for tileid_str in subsamp_dict.keys():
    
    # tileid_str = '80605'
    tileid = int(tileid_str)

    for subset_index, subset in enumerate(subsamp_dict[tileid_str]):
        # print(subset)

        mask = np.in1d(cframes['expid'], subset)
        _, n_exp = np.unique(cframes['petal_loc'][mask], return_counts=True)
        n_exp = np.max(n_exp)
        petal_locs = np.arange(10)

        for petal_loc in petal_locs:

            mask = np.in1d(cframes['expid'], subset) & (cframes['petal_loc']==petal_loc)
            if np.sum(mask)!=n_exp:
                print('Tile {} subset {} petal {} has only {} out of {} exposures; skip'.format(tileid, subset_index+1, petal_loc, np.sum(mask), n_exp))
                continue

            spectra_output_loc = os.path.join('$OUTDIR', str(tileid), str(subset_index+1), 'spectra-{}-{}-1xsubset{}.fits'.format(petal_loc, tileid, subset_index+1))
            coadd_output_loc = os.path.join('$OUTDIR', str(tileid), str(subset_index+1), 'coadd-{}-{}-1xsubset{}.fits'.format(petal_loc, tileid, subset_index+1))
            
            write_text = 'desi_coadd_spectra --onetile -i {} -o {}\n'.format(spectra_output_loc, coadd_output_loc)
            f_all.write(write_text)

            if os.path.isfile(os.path.expandvars(coadd_output_loc)) and (not overwrite):
                # print('{} already exists'.format(coadd_output_loc))
                pass
            else:
                # print(write_text)
                f.write(write_text)

f_all.close()
f.close()

Tile 80605 subset 1 petal 3 has only 0 out of 2 exposures; skip
Tile 80605 subset 2 petal 3 has only 1 out of 2 exposures; skip
Tile 80605 subset 3 petal 3 has only 1 out of 2 exposures; skip
Tile 80605 subset 5 petal 3 has only 0 out of 2 exposures; skip
Tile 80606 subset 1 petal 3 has only 1 out of 2 exposures; skip
Tile 80610 subset 5 petal 3 has only 1 out of 2 exposures; skip
Tile 80678 subset 1 petal 3 has only 0 out of 2 exposures; skip
Tile 80678 subset 2 petal 3 has only 0 out of 2 exposures; skip
Tile 80678 subset 3 petal 3 has only 0 out of 2 exposures; skip
Tile 80678 subset 4 petal 3 has only 0 out of 2 exposures; skip
Tile 80678 subset 5 petal 3 has only 0 out of 2 exposures; skip
Tile 80690 subset 1 petal 3 has only 0 out of 2 exposures; skip
Tile 80690 subset 2 petal 3 has only 0 out of 3 exposures; skip
Tile 80690 subset 3 petal 3 has only 0 out of 3 exposures; skip
Tile 80690 subset 4 petal 3 has only 0 out of 3 exposures; skip
Tile 80690 subset 5 petal 3 has only 0 o

__Print redrock commands__  
Include 10-minute timeout

In [16]:
command_output_path_all = 'rrdesi_1x_all.txt'
command_output_path = 'rrdesi_1x.txt'  # commands for reruns (e.g. after failures)

f_all = open(command_output_path_all, 'w')
f = open(command_output_path, 'w')

for tileid_str in subsamp_dict.keys():

    tileid = int(tileid_str)
    
    for subset_index, subset in enumerate(subsamp_dict[tileid_str]):

        mask = np.in1d(cframes['expid'], subset)
        _, n_exp = np.unique(cframes['petal_loc'][mask], return_counts=True)
        n_exp = np.max(n_exp)
        petal_locs = np.arange(10)

        for petal_loc in petal_locs:
        
            mask = np.in1d(cframes['expid'], subset) & (cframes['petal_loc']==petal_loc)
            if np.sum(mask)!=n_exp:
                print('Tile {} subset {} petal {} has only {} out of {} exposures; skip'.format(tileid, subset_index+1, petal_loc, np.sum(mask), n_exp))
                continue

            coadd_output_loc = os.path.join('$OUTDIR', str(tileid), str(subset_index+1), 'coadd-{}-{}-1xsubset{}.fits'.format(petal_loc, tileid, subset_index+1))
            redrock_output_loc = os.path.join('$OUTDIR', str(tileid), str(subset_index+1), 'redrock-{}-{}-1xsubset{}.fits'.format(petal_loc, tileid, subset_index+1))
            rrdetails_output_loc = os.path.join('$OUTDIR', str(tileid), str(subset_index+1), 'rrdetails-{}-{}-1xsubset{}.h5'.format(petal_loc, tileid, subset_index+1))
            log_loc = os.path.join('$OUTDIR', str(tileid), str(subset_index+1), 'logs/redrock-{}-{}-1xsubset{}.log'.format(petal_loc, tileid, subset_index+1))

            write_text = 'srun -N 1 -n 32 -c 2 -t 00:10:00 rrdesi_mpi -i {} -o {} --details {} &> {}\n'.format(coadd_output_loc, redrock_output_loc, rrdetails_output_loc, log_loc)
            f_all.write(write_text)
        
            if os.path.isfile(os.path.expandvars(redrock_output_loc)) and (not overwrite):
                # print('{} already exists'.format(redrock_output_loc))
                pass
            else:
                # print(write_text)
                f.write(write_text)
f.close()
f_all.close()

Tile 80605 subset 1 petal 3 has only 0 out of 2 exposures; skip
Tile 80605 subset 2 petal 3 has only 1 out of 2 exposures; skip
Tile 80605 subset 3 petal 3 has only 1 out of 2 exposures; skip
Tile 80605 subset 5 petal 3 has only 0 out of 2 exposures; skip
Tile 80606 subset 1 petal 3 has only 1 out of 2 exposures; skip
Tile 80610 subset 5 petal 3 has only 1 out of 2 exposures; skip
Tile 80678 subset 1 petal 3 has only 0 out of 2 exposures; skip
Tile 80678 subset 2 petal 3 has only 0 out of 2 exposures; skip
Tile 80678 subset 3 petal 3 has only 0 out of 2 exposures; skip
Tile 80678 subset 4 petal 3 has only 0 out of 2 exposures; skip
Tile 80678 subset 5 petal 3 has only 0 out of 2 exposures; skip
Tile 80690 subset 1 petal 3 has only 0 out of 2 exposures; skip
Tile 80690 subset 2 petal 3 has only 0 out of 3 exposures; skip
Tile 80690 subset 3 petal 3 has only 0 out of 3 exposures; skip
Tile 80690 subset 4 petal 3 has only 0 out of 3 exposures; skip
Tile 80690 subset 5 petal 3 has only 0 o

__Print QSO MgII afterburner commands__  

In [17]:
command_output_path_all = 'qso_mgii_1x_all.txt'
command_output_path = 'qso_mgii_1x.txt'  # commands for reruns (e.g. after failures)

f_all = open(command_output_path_all, 'w')
f = open(command_output_path, 'w')

for tileid_str in subsamp_dict.keys():
# for tileid_str in ['80605', '80607', '80609']:  # LRG+QSO tiles only

    tileid = int(tileid_str)
    
    for subset_index, subset in enumerate(subsamp_dict[tileid_str]):

        mask = np.in1d(cframes['expid'], subset)
        _, n_exp = np.unique(cframes['petal_loc'][mask], return_counts=True)
        n_exp = np.max(n_exp)
        petal_locs = np.arange(10)
        
        for petal_loc in petal_locs:
        
            mask = np.in1d(cframes['expid'], subset) & (cframes['petal_loc']==petal_loc)
            if np.sum(mask)!=n_exp:
                print('Tile {} subset {} petal {} has only {} out of {} exposures; skip'.format(tileid, subset_index+1, petal_loc, np.sum(mask), n_exp))
                continue

            coadd_output_loc = os.path.join('$OUTDIR', str(tileid), str(subset_index+1), 'coadd-{}-{}-1xsubset{}.fits'.format(petal_loc, tileid, subset_index+1))
            redrock_output_loc = os.path.join('$OUTDIR', str(tileid), str(subset_index+1), 'redrock-{}-{}-1xsubset{}.fits'.format(petal_loc, tileid, subset_index+1))
            qso_mgii_output_loc = os.path.join('$OUTDIR', str(tileid), str(subset_index+1), 'qso_mgii-{}-{}-1xsubset{}.fits'.format(petal_loc, tileid, subset_index+1))
            log_loc = os.path.join('$OUTDIR', str(tileid), str(subset_index+1), 'logs/qso_mgii-{}-{}-1xsubset{}.log'.format(petal_loc, tileid, subset_index+1))

            write_text = 'srun -N 1 -n 1 -c 32 desi_qso_mgii_afterburner --coadd {} --redrock {} --output {} --target_selection all_targets --save_target all &> {}\n'.format(coadd_output_loc, redrock_output_loc, qso_mgii_output_loc, log_loc)
            f_all.write(write_text)
        
            if os.path.isfile(os.path.expandvars(qso_mgii_output_loc)) and (not overwrite):
                # print('{} already exists'.format(qso_mgii_output_loc))
                pass
            else:
                # print(write_text)
                f.write(write_text)
f.close()
f_all.close()

Tile 80605 subset 1 petal 3 has only 0 out of 2 exposures; skip
Tile 80605 subset 2 petal 3 has only 1 out of 2 exposures; skip
Tile 80605 subset 3 petal 3 has only 1 out of 2 exposures; skip
Tile 80605 subset 5 petal 3 has only 0 out of 2 exposures; skip
Tile 80606 subset 1 petal 3 has only 1 out of 2 exposures; skip
Tile 80610 subset 5 petal 3 has only 1 out of 2 exposures; skip
Tile 80678 subset 1 petal 3 has only 0 out of 2 exposures; skip
Tile 80678 subset 2 petal 3 has only 0 out of 2 exposures; skip
Tile 80678 subset 3 petal 3 has only 0 out of 2 exposures; skip
Tile 80678 subset 4 petal 3 has only 0 out of 2 exposures; skip
Tile 80678 subset 5 petal 3 has only 0 out of 2 exposures; skip
Tile 80690 subset 1 petal 3 has only 0 out of 2 exposures; skip
Tile 80690 subset 2 petal 3 has only 0 out of 3 exposures; skip
Tile 80690 subset 3 petal 3 has only 0 out of 3 exposures; skip
Tile 80690 subset 4 petal 3 has only 0 out of 3 exposures; skip
Tile 80690 subset 5 petal 3 has only 0 o

__Print QSO QN afterburner commands__  

In [18]:
command_output_path_all = 'qso_qn_1x_all.txt'
command_output_path = 'qso_qn_1x.txt'  # commands for reruns (e.g. after failures)

f_all = open(command_output_path_all, 'w')
f = open(command_output_path, 'w')

for tileid_str in subsamp_dict.keys():
# for tileid_str in ['80605', '80607', '80609']:  # LRG+QSO tiles only

    tileid = int(tileid_str)
    
    for subset_index, subset in enumerate(subsamp_dict[tileid_str]):

        mask = np.in1d(cframes['expid'], subset)
        _, n_exp = np.unique(cframes['petal_loc'][mask], return_counts=True)
        n_exp = np.max(n_exp)
        petal_locs = np.arange(10)
        
        for petal_loc in petal_locs:
        
            mask = np.in1d(cframes['expid'], subset) & (cframes['petal_loc']==petal_loc)
            if np.sum(mask)!=n_exp:
                print('Tile {} subset {} petal {} has only {} out of {} exposures; skip'.format(tileid, subset_index+1, petal_loc, np.sum(mask), n_exp))
                continue

            coadd_output_loc = os.path.join('$OUTDIR', str(tileid), str(subset_index+1), 'coadd-{}-{}-1xsubset{}.fits'.format(petal_loc, tileid, subset_index+1))
            redrock_output_loc = os.path.join('$OUTDIR', str(tileid), str(subset_index+1), 'redrock-{}-{}-1xsubset{}.fits'.format(petal_loc, tileid, subset_index+1))
            qso_qn_output_loc = os.path.join('$OUTDIR', str(tileid), str(subset_index+1), 'qso_qn-{}-{}-1xsubset{}.fits'.format(petal_loc, tileid, subset_index+1))
            log_loc = os.path.join('$OUTDIR', str(tileid), str(subset_index+1), 'logs/qso_qn-{}-{}-1xsubset{}.log'.format(petal_loc, tileid, subset_index+1))

            write_text = 'srun -N 1 -n 1 -c 32 desi_qso_qn_afterburner --coadd {} --redrock {} --output {} --target_selection all_targets --save_target all &> {}\n'.format(coadd_output_loc, redrock_output_loc, qso_qn_output_loc, log_loc)
            f_all.write(write_text)
        
            if os.path.isfile(os.path.expandvars(qso_qn_output_loc)) and (not overwrite):
                # print('{} already exists'.format(qso_qn_output_loc))
                pass
            else:
                # print(write_text)
                f.write(write_text)
f.close()
f_all.close()

Tile 80605 subset 1 petal 3 has only 0 out of 2 exposures; skip
Tile 80605 subset 2 petal 3 has only 1 out of 2 exposures; skip
Tile 80605 subset 3 petal 3 has only 1 out of 2 exposures; skip
Tile 80605 subset 5 petal 3 has only 0 out of 2 exposures; skip
Tile 80606 subset 1 petal 3 has only 1 out of 2 exposures; skip
Tile 80610 subset 5 petal 3 has only 1 out of 2 exposures; skip
Tile 80678 subset 1 petal 3 has only 0 out of 2 exposures; skip
Tile 80678 subset 2 petal 3 has only 0 out of 2 exposures; skip
Tile 80678 subset 3 petal 3 has only 0 out of 2 exposures; skip
Tile 80678 subset 4 petal 3 has only 0 out of 2 exposures; skip
Tile 80678 subset 5 petal 3 has only 0 out of 2 exposures; skip
Tile 80690 subset 1 petal 3 has only 0 out of 2 exposures; skip
Tile 80690 subset 2 petal 3 has only 0 out of 3 exposures; skip
Tile 80690 subset 3 petal 3 has only 0 out of 3 exposures; skip
Tile 80690 subset 4 petal 3 has only 0 out of 3 exposures; skip
Tile 80690 subset 5 petal 3 has only 0 o

__Print emission line afterburner commands__  

In [19]:
command_output_path_all = 'desi_emlinefit_afterburner_1x_all.txt'
command_output_path = 'desi_emlinefit_afterburner_1x.txt'  # commands for reruns (e.g. after failures)

f_all = open(command_output_path_all, 'w')
f = open(command_output_path, 'w')

for tileid_str in subsamp_dict.keys():

    tileid = int(tileid_str)

    for subset_index, subset in enumerate(subsamp_dict[tileid_str]):

        mask = np.in1d(cframes['expid'], subset)
        _, n_exp = np.unique(cframes['petal_loc'][mask], return_counts=True)
        n_exp = np.max(n_exp)
        petal_locs = np.arange(10)

        for petal_loc in petal_locs:

            mask = np.in1d(cframes['expid'], subset) & (cframes['petal_loc']==petal_loc)
            if np.sum(mask)!=n_exp:
                print('Tile {} subset {} petal {} has only {} out of {} exposures; skip'.format(tileid, subset_index+1, petal_loc, np.sum(mask), n_exp))
                continue

            coadd_output_loc = os.path.join('$OUTDIR', str(tileid), str(subset_index+1), 'coadd-{}-{}-1xsubset{}.fits'.format(petal_loc, tileid, subset_index+1))
            redrock_output_loc = os.path.join('$OUTDIR', str(tileid), str(subset_index+1), 'redrock-{}-{}-1xsubset{}.fits'.format(petal_loc, tileid, subset_index+1))
            emline_output_loc = os.path.join('$OUTDIR', str(tileid), str(subset_index+1), 'emline-{}-{}-1xsubset{}.fits'.format(petal_loc, tileid, subset_index+1))
            log_loc = os.path.join('$OUTDIR', str(tileid), str(subset_index+1), 'logs/emline-{}-{}-1xsubset{}.log'.format(petal_loc, tileid, subset_index+1))

            write_text = 'desi_emlinefit_afterburner --redrock {} --coadd {} --output {} &> {}\n'.format(redrock_output_loc, coadd_output_loc, emline_output_loc, log_loc)
            f_all.write(write_text)

            if os.path.isfile(os.path.expandvars(emline_output_loc)) and (not overwrite):
                # print('{} already exists'.format(emline_output_loc))
                pass
            else:
                # print(write_text)
                f.write(write_text)

f_all.close()
f.close()

Tile 80605 subset 1 petal 3 has only 0 out of 2 exposures; skip
Tile 80605 subset 2 petal 3 has only 1 out of 2 exposures; skip
Tile 80605 subset 3 petal 3 has only 1 out of 2 exposures; skip
Tile 80605 subset 5 petal 3 has only 0 out of 2 exposures; skip
Tile 80606 subset 1 petal 3 has only 1 out of 2 exposures; skip
Tile 80610 subset 5 petal 3 has only 1 out of 2 exposures; skip
Tile 80678 subset 1 petal 3 has only 0 out of 2 exposures; skip
Tile 80678 subset 2 petal 3 has only 0 out of 2 exposures; skip
Tile 80678 subset 3 petal 3 has only 0 out of 2 exposures; skip
Tile 80678 subset 4 petal 3 has only 0 out of 2 exposures; skip
Tile 80678 subset 5 petal 3 has only 0 out of 2 exposures; skip
Tile 80690 subset 1 petal 3 has only 0 out of 2 exposures; skip
Tile 80690 subset 2 petal 3 has only 0 out of 3 exposures; skip
Tile 80690 subset 3 petal 3 has only 0 out of 3 exposures; skip
Tile 80690 subset 4 petal 3 has only 0 out of 3 exposures; skip
Tile 80690 subset 5 petal 3 has only 0 o

__Create directories for logs__

In [20]:
from pathlib import Path

for tileid_str in subsamp_dict.keys():

    tileid = int(tileid_str)
    
    for subset_index, subset in enumerate(subsamp_dict[tileid_str]):
        
        log_dir = os.path.join('$OUTDIR', str(tileid), str(subset_index+1), 'logs')
        Path(os.path.expandvars(log_dir)).mkdir(parents=True, exist_ok=True)